In [ ]:
import json

data = {
    "intents": [
        {"tag": "greeting",
         "patterns": ["Hi", "Hello", "How are you?", "Good day", "Hey"],
         "responses": ["Hello!", "Hi there!", "Greetings!", "How can I assist you?"]
        },
        {"tag": "goodbye",
         "patterns": ["Bye", "See you", "Goodbye", "Catch you later"],
         "responses": ["Goodbye!", "See you soon!", "Have a nice day!"]
        },
        {"tag": "thanks",
         "patterns": ["Thanks", "Thank you", "That's helpful", "Thanks a lot"],
         "responses": ["You're welcome!", "Happy to help!", "Anytime!"]
        },
        {"tag": "hours",
         "patterns": ["What are your hours?", "When are you open?", "Opening hours"],
         "responses": ["We are open 9am to 5pm, Monday to Friday."]
        },
        {"tag": "payments",
         "patterns": ["Do you accept credit cards?", "What payment methods do you support?"],
         "responses": ["We accept Visa, Mastercard and PayPal."]
        },
        {"tag": "fallback",
         "patterns": [],
         "responses": ["Sorry, I didn't understand that. Can you rephrase?"]
        }
    ]
}

# Save dataset
with open("intents.json", "w") as f:
    json.dump(data, f, indent=4)


In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string
import json

# Download required NLTK data (run once)
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Tokenize
    tokens = nltk.word_tokenize(text.lower())
    # Remove punctuation and stopwords, lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words and word not in string.punctuation]
    return tokens

# Example usage
print(preprocess_text("Hello! How are you doing today?"))


In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string
import json

# Download required NLTK data (run once)
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Tokenize
    tokens = nltk.word_tokenize(text.lower())
    # Remove punctuation and stopwords, lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words and word not in string.punctuation]
    return tokens

# Example usage
print(preprocess_text("Hello! How are you doing today?"))


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Load data
with open('intents.json') as file:
    intents = json.load(file)

# Extract patterns and tags
patterns = []
tags = []

for intent in intents['intents']:
    for pattern in intent['patterns']:
        patterns.append(pattern)
        tags.append(intent['tag'])

# Preprocess all patterns
processed_patterns = [' '.join(preprocess_text(p)) for p in patterns]

# TF-IDF Vectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(processed_patterns)

print(X.shape)  # (number of patterns, number of features)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, tags, test_size=0.2, random_state=42)

# Train classifier
clf = LogisticRegression()
clf.fit(X_train, y_train)

# Test accuracy
y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))


In [ ]:
import random

def chatbot_response(text):
    # Preprocess and vectorize input text
    text_processed = ' '.join(preprocess_text(text))
    text_vec = vectorizer.transform([text_processed])
    
    # Predict intent
    pred_tag = clf.predict(text_vec)[0]
    
    # If confidence is low, fallback
    prob = clf.predict_proba(text_vec).max()
    if prob < 0.4:  # threshold for fallback
        pred_tag = "fallback"
    
    # Get response
    for intent in intents['intents']:
        if intent['tag'] == pred_tag:
            return random.choice(intent['responses'])

# Try chatbot
print(chatbot_response("Hey there!"))
print(chatbot_response("What time do you open?"))
print(chatbot_response("Can you do my homework?"))  # Should fallback


In [ ]:
import random

def chatbot_response(text):
    # Preprocess and vectorize input text
    text_processed = ' '.join(preprocess_text(text))
    text_vec = vectorizer.transform([text_processed])
    
    # Predict intent
    pred_tag = clf.predict(text_vec)[0]
    
    # If confidence is low, fallback
    prob = clf.predict_proba(text_vec).max()
    if prob < 0.4:  # threshold for fallback
        pred_tag = "fallback"
    
    # Get response
    for intent in intents['intents']:
        if intent['tag'] == pred_tag:
            return random.choice(intent['responses'])

# Try chatbot
print(chatbot_response("Hey there!"))
print(chatbot_response("What time do you open?"))
print(chatbot_response("Can you do my homework?"))  # Should fallback


In [ ]:
print("Chatbot is running! Type 'quit' to exit.")

while True:
    user_input = input("You: ")
    if user_input.lower() == 'quit':
        print("Chatbot: Goodbye!")
        break
    response = chatbot_response(user_input)
    print(f"Chatbot: {response}")


In [ ]:
import spacy

# Download the English model if not installed: python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")

def spacy_preprocess(text):
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    return tokens

# Example
print(spacy_preprocess("Hello! How are you doing today?"))


In [ ]:
!pip install transformers sentence-transformers

from sentence_transformers import SentenceTransformer
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

# Load pre-trained SentenceTransformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Prepare training data (patterns and tags)
with open('intents.json') as f:
    intents = json.load(f)

patterns = []
tags = []
for intent in intents['intents']:
    for pattern in intent['patterns']:
        patterns.append(pattern)
        tags.append(intent['tag'])

# Get embeddings for each pattern
embeddings = model.encode(patterns)

# Train a KNN classifier (works well with embeddings)
knn = KNeighborsClassifier(n_neighbors=3, metric='cosine')
knn.fit(embeddings, tags)

# Predict intent function
def predict_intent_bert(text):
    emb = model.encode([text])
    distances, indices = knn.kneighbors(emb)
    pred_tag = knn.predict(emb)[0]
    confidence = 1 - distances[0][0]  # cosine similarity as confidence

    # Fallback threshold
    if confidence < 0.5:
        pred_tag = "fallback"
    
    return pred_tag, confidence

# Test it
tag, conf = predict_intent_bert("Can you tell me your opening hours?")
print(f"Predicted intent: {tag}, Confidence: {conf:.2f}")


In [ ]:
def extract_entities(text):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

print(extract_entities("I want to book a flight to New York next Monday"))


In [ ]:
class ChatbotContext:
    def __init__(self):
        self.context = {}
    
    def update(self, intent, entities):
        self.context['last_intent'] = intent
        self.context['last_entities'] = entities
    
    def get_context(self):
        return self.context

context = ChatbotContext()

def chatbot_response_advanced(text):
    intent, confidence = predict_intent_bert(text)
    entities = extract_entities(text)
    
    # Update context
    context.update(intent, entities)

    # Choose response
    for intent_data in intents['intents']:
        if intent_data['tag'] == intent:
            response = np.random.choice(intent_data['responses'])
            break
    else:
        response = "Sorry, I didn't get that."

    # Add entities info to response for demonstration
    if entities:
        response += f" (I noticed you mentioned: {entities})"
    
    return response

# Try chat
print(chatbot_response_advanced("Hi there!"))
print(chatbot_response_advanced("Can you tell me your working hours?"))
print(chatbot_response_advanced("I want to pay with a credit card."))
print(chatbot_response_advanced("Book a flight to Paris next Friday."))


In [ ]:
pip install streamlit


In [ ]:
import streamlit as st

st.title("Advanced AI Chatbot")

context = ChatbotContext()

user_input = st.text_input("You:")

if user_input:
    response = chatbot_response_advanced(user_input)
    st.text_area("Chatbot:", value=response, height=150)


In [ ]:
streamlit run app.py


In [ ]:
!streamlit run app.py


In [ ]:
import joblib

joblib.dump(knn, "knn_intent_model.joblib")
joblib.dump(model, "sentence_transformer_model.joblib")


In [ ]:
knn = joblib.load("knn_intent_model.joblib")
model = joblib.load("sentence_transformer_model.joblib")


In [ ]:
import random

def chatbot_response(text):
    # Preprocess and vectorize input text
    text_processed = ' '.join(preprocess_text(text))
    text_vec = vectorizer.transform([text_processed])
    
    # Predict intent
    pred_tag = clf.predict(text_vec)[0]
    
    # If confidence is low, fallback
    prob = clf.predict_proba(text_vec).max()
    if prob < 0.4:  # threshold for fallback
        pred_tag = "fallback"
    
    # Get response
    for intent in intents['intents']:
        if intent['tag'] == pred_tag:
            return random.choice(intent['responses'])

# Try chatbot
print(chatbot_response("Hey there!"))
print(chatbot_response("What time do you open?"))
print(chatbot_response("Can you do my homework?"))  # Should fallback
